# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import numpy as np
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from tdoa_py import correlation, simulation_room

In [2]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.sheet1

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1)
df_simulations.head() # Verificar el resultado



,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,src_az_deg,src_z,theta_1,theta_2,theta_3,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,sim1,classic,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,15,1.2,,,,18.499357,0.000188,18.499357,201.726984,"(100, 100, 100)"


In [3]:
# df_simulations = pd.read_csv('simulations.csv')
# df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1)
# df_simulations.head() # Verificar el resultado

In [4]:
for sim in df_simulations.itertuples():
    angle, wavs = simulation_room.sim_room_Nmics(
        wav_path = f'audios/anechoic/{sim.audio}',
        out_dir = f'audios/output/{sim.sim_name}',
        fs=48000,

        room_dim=sim.room_dim,
        rt60=sim.rt60,
        snr_db=sim.snr_db,

        n_mics=sim.n_mics,
        mic_d=sim.mic_d,
        mic_z=sim.mic_z,
        mic_directivity=sim.mic_directivity,

        src_dist=sim.src_dist,
        src_az_deg=sim.src_az_deg,
        src_z=sim.src_z
    )
    
    avg_angle_deg, avg_tdoa, hemi_avgs, tdoas, angles = correlation.estimate_doa_from_wavs(
    wav_folder = f'audios/output/{sim.sim_name}',
    mic_d = sim.mic_d,
    method = sim.method,
    fs = 48000
    )
    
    print(angles)
    df_simulations['avg_angle_deg'] = avg_angle_deg
    df_simulations['avg_tdoa'] = avg_tdoa
    df_simulations['hemi_1'] = hemi_avgs['H1']
    df_simulations['hemi_2'] = hemi_avgs['H2']
    df_simulations['theta_1'] = angles[0]
    df_simulations['theta_2'] = angles[1]
    df_simulations['theta_3'] = angles[2]
    

# Convertir el DataFrame a una lista de listas
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")
    


Fuente en: x=98.30, y=62.94, z=1.20
Mic 0: TDOA = 0.000562 s, Ángulo estimado = 15.27°
Mic 1: TDOA = 0.000271 s, Ángulo estimado = 21.73°
Mic 3: TDOA = -0.000271 s, Ángulo estimado = 201.73°

Hemisferio dominante: H1
H1: Promedio = 18.50°
H2: Promedio = 201.73°
[np.float64(15.271729393963701), np.float64(21.726983901452964), np.float64(201.72698390145297)]
DataFrame guardado correctamente en Google Sheets.
